<a href="https://colab.research.google.com/github/nagaokayama/captioning_scripts/blob/main/Text_cordinator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 0. google driveのファイルを使用する場合はまずこのスクリプトを実行してください {display-mode: "form"}
# @markdown ←●に▶が入ってるマークが「セルを実行ボタン」です。これを押して実行\
# @markdown グーグルドライブの中身はファイルブラウザのgdrive/MyDriveに共有されておりここから閲覧できます

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import re
import os
import glob

# @title 1. 画像情報の抽出 {display-mode: "form"}

# This code will be hidden when the notebook is loaded.


# @markdown 右端にあるフォルダのマークをクリックしてファイルブラウザを開き\
# @markdown 目的のフォルダを右クリックして「パスをコピー」でファイルパスをコピーできます

# @markdown 画像の入っているフォルダを指定
image_folder = "/content/Untitled Folder" #@param {type:"string"}
# @markdown 「セルを実行」ボタンを押してテキストファイルを生成\
# @markdown テキストファイルはimage_folderに指定したフォルダに作成されます。

# @markdown ---

 #フォルダとテキストのファイル名は自動で決める

def extract_fileName(imagesFolder):
  # 提出可能な画像の拡張子は？
  pattern = re.compile(".*\.(png|jpg|jpeg|gif|webp|tiff)$",re.IGNORECASE)
  tmpFiles = glob.glob(f"{imagesFolder}/*.*")
  fileNameList = []

  for name in tmpFiles:
    # 画像ファイル以外の拡張子を持つファイルを除外
    basename = os.path.basename(name)
    # print(basename)
    ismatch = pattern.search(basename)
    if ismatch:
      fileNameList.append(ismatch.group(0))

  return fileNameList


def export_ImageFileName(textstring):
  text = ""
  for i in textstring:
    tmp = i.split(".")[0] # 拡張子を削除
    text += tmp+"\n"

  text = text.rstrip("\n") # 末尾の改行記号を削除
  print(text)
  filepath = os.path.join(image_folder,"imageFileList.txt")
  with open(filepath, mode = "w") as f:
    f.write(text)
  return 1

# 関数を実行
tmp = extract_fileName(image_folder)
export_ImageFileName(tmp)

以下開発用のメモなど


---



In [ ]:
# @title キャプション用テキストファイルの書き出し

# @markdown テキストファイルの書き出し先
export_folder = "" #@param {type:"string"}

with open()as f:

  f.write()

In [ ]:
# @title フォームのついたコードセルの書き方
# @markdown Forms support many types of fields.

no_type_checking = ''  # @param
string_type = 'example'  # @param {type: "string"}
slider_value = 141  # @param {type: "slider", min: 100, max: 200}
number = 102  # @param {type: "number"}
date = '2010-11-05'  # @param {type: "date"}
pick_me = "monday"  # @param ['monday', 'tuesday', 'wednesday', 'thursday']
select_or_input = "apples" # @param ["apples", "bananas", "oranges"] {allow-input: true}
# @markdown ---
